In [1]:
import h5py
import numpy as np
from sklearn.utils import shuffle
np.random.seed(2017)

X_train = []
X_test = []

for filename in ["gap_ResNet50.h5", "gap_Xception.h5", "gap_InceptionV3.h5"]:
    with h5py.File(filename, 'r') as h:
        X_train.append(np.array(h['train']))
        X_test.append(np.array(h['test']))
        y_train = np.array(h['label'])

X_train = np.concatenate(X_train, axis=1)
X_test = np.concatenate(X_test, axis=1)

X_train, y_train = shuffle(X_train, y_train)

AxisError: axis 1 is out of bounds for array of dimension 1

In [3]:
from keras.models import *
from keras.layers import *

input_tensor = Input(X_train.shape[1:])
x = input_tensor
x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid')(x)
model = Model(input_tensor, x)

model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

Using TensorFlow backend.
W0418 16:09:15.990686 11684 deprecation.py:506] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3721: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0418 16:09:16.001693 11684 deprecation.py:237] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4421: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0418 16:09:16.023681 11684 deprecation.py:237] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3645: The name tf.log is deprecated. Please use tf.math.log instead.



In [4]:
from IPython.display import SVG
from keras.utils.visualize_util import model_to_dot, plot

SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))

ModuleNotFoundError: No module named 'keras.utils.visualize_util'

In [5]:
model.fit(X_train, y_train, batch_size=128, nb_epoch=8, validation_split=0.2)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Train on 3222 samples, validate on 806 samples
Epoch 1/8
3222/3222 [==============================] - 1s 165us/step - loss: 0.2711 - acc: 0.8799 - val_loss: 0.0842 - val_acc: 0.9901
Epoch 2/8
3222/3222 [==============================] - 0s 72us/step - loss: 0.0647 - acc: 0.9867 - val_loss: 0.0528 - val_acc: 0.9888
Epoch 3/8
3222/3222 [==============================] - 0s 70us/step - loss: 0.0413 - acc: 0.9910 - val_loss: 0.0381 - val_acc: 0.9901
Epoch 4/8
3222/3222 [==============================] - 0s 71us/step - loss: 0.0343 - acc: 0.9919 - val_loss: 0.0323 - val_acc: 0.9901
Epoch 5/8
3222/3222 [==============================] - 0s 71us/step - loss: 0.0243 - acc: 0.9947 - val_loss: 0.0317 - val_acc: 0.9926
Epoch 6/8
3222/3222 [==============================] - 0s 71us/step - loss: 0.0204 - acc: 0.9947 - val_loss: 0.0303 - val_acc: 0.9913
Epoch 7/8
3222/3222 [==============================] - 0s 72us/step - loss: 0.0158 - acc: 0.9966 - val_loss: 0.0231 - val_acc: 0.9938
Epoch 8/8
3222

In [6]:
model.save('model.h5')

In [7]:
y_pred = model.predict(X_test, verbose=1)
y_pred = y_pred.clip(min=0.005, max=0.995)

ValueError: Error when checking input: expected input_1 to have shape (6144,) but got array with shape (1,)

In [8]:
import pandas as pd
from keras.preprocessing.image import *

df = pd.read_csv("sample_submission.csv")

image_size = (224, 224)
gen = ImageDataGenerator()
test_generator = gen.flow_from_directory("test2", image_size, shuffle=False, 
                                         batch_size=16, class_mode=None)

for i, fname in enumerate(test_generator.filenames):
    index = int(fname[fname.rfind('/')+1:fname.rfind('.')])
    df.set_value(index-1, 'label', y_pred[i])

df.to_csv('pred.csv', index=None)
df.head(10)

Found 0 images belonging to 1 classes.


,id,label
0,1,0.5
1,2,0.5
2,3,0.5
3,4,0.5
4,5,0.5
5,6,0.5
6,7,0.5
7,8,0.5
8,9,0.5
9,10,0.5
